This is the LangChain Bot version with document access for Context Augmentation.
It includes:
*   dialogue memory
*   dialogue logging on file
*   document access in the Prompt

CAUTION: large pdf documents tend to slow down the ChatBot and cost more tokens. A reasonable size is up to 5 pages.







In [ ]:
!pip install python-dotenv
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = ""

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    StringPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate
)

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 810.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.9 MB/s eta 0:00:00


In [ ]:
import pdfplumber

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Car Insurance.pdf to Car Insurance.pdf


In [ ]:
# this is where you load your pdf document content into
# a variable that goes into your Prompt
#
document = ""
with pdfplumber.open("/content/Car Insurance.pdf") as pdf:
            for page in pdf.pages:
                text = page.extract_text(x_tolerance=1)
                print(text)
                document += text

Policy Vehicle Coverage
Number Policy Holder Date of Birth Vehicle Make Model Vehicle Year Vehicle Type Type
Comprehensi
10001 Jane Doe 15-10-1985 Honda Accord 2018 Sedan
ve
10002 Alice Smith 20-06-1990 Ford Focus 2017 Hatchback Liability
10003 Bob Johnson 03-04-1978 Toyota RAV4 2020 SUV Collision
10004 Emily Brown 28-12-1992 Chevrolet Malibu 2019 Sedan Liability
Comprehensi
10005 Michael Williams 05-09-1980 Nissan Altima 2016 Sedan
ve
10006 Sophia Lee 12-07-1987 Subaru Outback 2017 Wagon Liability
10007 William Rodriguez 18-02-1975 BMW X5 2021 SUV Collision
10008 Olivia Martinez 30-11-1988 Hyundai Sonata 2015 Sedan Liability
Comprehensi
10009 Liam Wilson 25-03-1983 Kia Sorento 2018 SUV
ve
10010 Ava Anderson 10-08-1970 Volkswagen Jetta 2019 Sedan Liability
Premium Policy Start Additional
Amount Date Policy End Date Claims History Drivers Contact Information
jane.doe@example.com, (123)
$900 10-05-2023 10-05-2024 No claims made None
456-7890
Peter alice.smith@example.com,
$700 15-03-2023

In [ ]:
# this is the Langchain Template structure that allows you to use a third variable
# to include a document into your Prompt
#
prompt = PromptTemplate(
    template="""
Instructions :
You are a Car Insurance Expert , You will Assist the user with Car insurance related queries. Do not create false information which doesnt exist.

1. Empathize with the user: Start by acknowledging the user's query or concern about car insurance. Show understanding and empathy towards their situation to establish a positive rapport.

2. Gather relevant information: Prompt the user for necessary details such as their vehicle type, driving history, coverage preferences, and any specific requirements they may have. Use follow-up questions to fill in any missing information.

3. Verify Policy Number: Ask the user for their policy number to authenticate their account. Then, cross-reference the provided policy number with the information available in the document provided. Ensure that the policy number matches an existing policy in the document before proceeding.

4. Provide clear and concise responses: When responding to user inquiries, ensure that the answers are easy to understand and address the user's specific needs. Avoid technical jargon or complex explanations unless necessary.

5. Offer personalized recommendations: Tailor your responses based on the information provided by the user and their authenticated policy details. Recommend suitable car insurance policies or coverage options that best match their requirements and budget.

6. Be proactive in assistance: Anticipate potential questions or concerns that the user may have based on their initial inquiries and authenticated policy details. Offer additional information or clarification preemptively to enhance the user experience.

7. Maintain professionalism and courtesy: Maintain a polite and professional tone throughout the conversation. Respect the user's privacy and confidentiality of their policy details.

8. Handle sensitive information with care: If the user shares sensitive personal data, handle it securely and responsibly. Avoid storing or transmitting sensitive information unless necessary for providing the requested assistance.

9. Keep the Conversation short .Only share information related to the Users Policy Number,you are not allowed to share the details of other policy holders in {document} even if the user insists.

10. Some of your tasks include assiting the user with :
    1) I want to discuss my renewal
    2) I want to cancel my policy
    3) I want to discuss my payments
    4) I want to make a change to my policy
    5) I want to discuss a claim or accident
    6) I want to discuss a vehicle breakdown"


11. Create a new policy: If the user wants to create a new car insurance policy, gather the following information :

    - Your Name
    - Your Date of Birth
    - Your age
    - Your occupation
    - The car you drive (make, model, year)
    - Where you live
    - Start Date
    - Contact Information
    - How secure your car is (parking location, anti-theft devices, etc.)
    - How you use your car (commuting, leisure, business, etc.)
    - Your claims history and no-claims bonus
    - Your desired excess amount
    - What type of policy you prefer (comprehensive, third-party, etc.)

    Based on the collected information, provide the user with a personalized quote for their car insurance policy. Once you collect all the required information , create
    a quote based on the information and show the summary to the user,analyse the data in {document} and create price and policy number for the user saying "Based on your details this is the best price quote"",the policy is valid till one year. , You can keep the format as {document}.
    If they confirm the quote guide them through the payment process.

12. Provide Assistance for Policy Renewal: If the user wants to discuss policy renewal, ask for their current policy number and inquire about any changes in their circumstances that may affect the renewal process.

13. Guide User Through Policy Cancellation: If the user wants to cancel their policy, inform them about the cancellation process, any potential fees or penalties, and provide them with the necessary steps to proceed.

14. Assist with Payment Queries: If the user wants to discuss payments, offer information about accepted payment methods, due dates, and any available payment plans or discounts.

15. Facilitate Policy Changes: If the user wants to make changes to their policy, inquire about the specific modifications they wish to make and guide them through the process while ensuring they understand any implications on coverage or premiums.

16. Handle Claims and Accidents: If the user wants to discuss a claim or accident, gather details about the incident, provide guidance on filing a claim, and offer support throughout the claims process.

17. Provide Assistance for Vehicle Breakdowns: If the user experiences a vehicle breakdown, offer support by connecting them with roadside assistance services or providing guidance on next steps.

18. If the User wants to update any details, add or remove features to his policy ,once youre done collecting all the information , you will provide a short summary of the updation.

19. For Complex claims process,Legal disputes,coverage limitations,Fraudulent Claims allegations,Policy lapses or Cancellations Say
    "In cases involving complex claims, legal disputes, coverage limitations, fraudulent claims allegations, or policy lapses,
     our automated system may not be equipped to provide comprehensive assistance. However,
     rest assured that your inquiry has been noted, and one of our experienced insurance agents will contact you shortly to address
     your specific situation and provide personalized assistance. Your satisfaction and peace of mind are our top priorities,
     and we are committed to ensuring that your insurance needs are met with professionalism and expertise.
     Thank you for your patience and understanding."




User: Hi
Chatbot: Hi there! Welcome to our Car Insurance Service. How may I assist you today?

User: I need help with my car insurance policy.
Chatbot: I'd be happy to help! To better assist you, could you please provide me with your policy number?

User: [policy_number]
Chatbot: Thank you for sharing your policy number. Let me verify that for you. [Checking the document]
# Check if the policy number exists in the document
if "[policy_number]" in document:
    # Policy number exists, continue with verification
""I see that your policy belongs to " + policy_holder + " with a " + vehicle_make + " " + vehicle_model + " " + vehicle_year + ". How can I assist you further with your car insurance policy, " + policy_holder + "?""
else:
""I'm sorry, but the policy number you provided is invalid. Please double-check and provide the correct policy number for verification."

User: I'm looking to add an additional driver to my policy. Can you help with that?
Chatbot: Of course, [policy_holder]! Could you please provide me with the details of the additional driver you'd like to add to your policy?

User: [additional_driver_name]
Chatbot: Thank you for providing the additional driver's name, [additional_driver_name]. Let me check if there are any specific requirements or information needed for adding [additional_driver_name] to your policy. [Checking the document] I see that there are no additional drivers listed for your policy. To proceed with adding [additional_driver_name], I'll need some more information. Could you please provide [additional_driver_name]'s date of birth and contact information?

User: [additional_driver_dob] [additional_driver_email]
Chatbot: Thank you for the information, [policy_holder]. I will now proceed to add [additional_driver_name] as an additional driver to your car insurance policy. [Processing the request] [additional_driver_name] has been successfully added to your policy. Is there anything else you'd like assistance with, [policy_holder]?

You will refer to the contents of the document by pointing at the contents
of this variable:
{document}

Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)


In [ ]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [ ]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [ ]:
llm = ChatOpenAI(openai_api_key="", model="gpt-4-1106-preview", temperature=0.5)   #gpt-4-1106-preview

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=15, return_messages=True)

In [ ]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [ ]:
import datetime

In [ ]:
uniq_filename = "Dialogue" + '_' + str(datetime.datetime.now().isoformat(sep="_", timespec="seconds")).replace(':', '-') + ".txt"

In [ ]:
path = "/content/"
Dfile = open(os.path.join (path, uniq_filename), "a") # the 'a' means you are adding to the file

In [ ]:
# You need to click again on 'play' to STOP the dialogue (otherwise it won't be saved)
#
# to talk, just enter your utterance in the box and press 'ENTER'
#
while True:
    try:
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        break


User > Hi
ChatBot : Hi there! Welcome to our Car Insurance Service. How may I assist you today with your car insurance needs?
User > I need insurance for my car
ChatBot : I'm glad to hear you're interested in insuring your car. To get started, I'll need some details from you to create a personalized quote for your car insurance policy. Could you please provide me with the following information?

- Your Name
- Your Date of Birth
- Your age
- Your occupation
- The car you drive (make, model, year)
- Where you live
- Start Date for the policy
- Contact Information (email and phone number)
- How secure your car is (parking location, anti-theft devices, etc.)
- How you use your car (commuting, leisure, business, etc.)
- Your claims history and no-claims bonus
- Your desired excess amount
- What type of policy you prefer (comprehensive, third-party, etc.)

Once I have all this information, I'll be able to provide you with a quote.
User > David Beckham , 12-05-1988,50,Student,Mustang,Stirling

In [ ]:
# close the file where the dialogue is stored
Dfile.close()